# custom monitoring template
---
By default, Disk usage and memory usage are not monitored by cloudwatch. This uses the scripts provided by AWS to add those features. http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/mon-scripts.html

STATUS: INCOMPLETE

In [2]:
from troposphere import Template, Ref, GetAtt, GetAZs, Base64, Join, FindInMap, Select, Output, Parameter, Tags
import troposphere.ec2 as ec2

In [3]:
resources = set()
t = Template()
t.add_version('2010-09-09')
t.add_description('Launch an instance with custom monitoring.')

# AZ selection
az1 = Select('0', GetAZs(''))
az2 = Select('1', GetAZs(''))
az3 = Select('2', GetAZs(''))

# Parameters
---

In [3]:
keyname = Parameter('keyname', 
                    Default='main', 
                    Description='Key to use for ssh access',
                    Type='String')
t.add_parameter(keyname);

# Maps

In [4]:
# Amazon Linux AMI 2016.09.0 (HVM), SSD Volume Type
t.add_mapping('RegionMap', { 'us-east-1':      {'ami': 'ami-b73b63a0'} # N. Virginia
                           , 'us-east-2':      {'ami': 'ami-58277d3d'} # Ohio
                           , 'us-west-1':      {'ami': 'ami-23e8a343'} # N. California
                           , 'us-west-2':      {'ami': 'ami-5ec1673e'} # Oregon
                           , 'eu-west-1':      {'ami': 'ami-9398d3e0'} # Ireland
                           , 'eu-central-1':   {'ami': 'ami-f9619996'} # Frankfurt
                           , 'ap-northeast-1': {'ami': 'ami-0c11b26d'} # Tokyo
                           , 'ap-northeast-2': {'ami': 'ami-983ce8f6'} # Seoul
                           , 'ap-southeast-1': {'ami': 'ami-b953f2da'} # Singapore
                           , 'ap-southeast-2': {'ami': 'ami-db704cb8'} # Sydney
                           , 'ap-south-1':     {'ami': 'ami-34b4c05b'} # Mumbai
                           , 'sa-east-1':      {'ami': 'ami-97831ffb'} # São Paulo 
                           });

# Resources
---

## Network

### VPC

In [5]:
vpc = ec2.VPC('exampleVpc', CidrBlock='10.0.0.0/16', Tags=rename_me)
igw = ec2.InternetGateway('igw', Tags=rename_me)
attach_igw = ec2.VPCGatewayAttachment('attachIgw', InternetGatewayId=Ref(igw), VpcId=Ref(vpc))

resources.add(vpc)
resources.add(igw)
resources.add(attach_igw)

### Subnets

In [6]:
public1 = ec2.Subnet('public1',
                     AvailabilityZone=az1,
                     CidrBlock='10.0.1.0/24',
                     VpcId=Ref(vpc),
                     Tags=rename_me)
public2 = ec2.Subnet('public2',
                     AvailabilityZone=az2,
                     CidrBlock='10.0.2.0/24',
                     VpcId=Ref(vpc),
                     Tags=rename_me)
public3 = ec2.Subnet('public3',
                     AvailabilityZone=az3,
                     CidrBlock='10.0.3.0/24',
                     VpcId=Ref(vpc),
                     Tags=rename_me)
resources.add(public1)
resources.add(public2)
resources.add(public3)

### Security Groups

In [7]:
allowed_ports = [80]
rules = [ec2.SecurityGroupRule(IpProtocol='tcp', CidrIp='0.0.0.0/0', FromPort=p, ToPort=p) for p in allowed_ports]

web_dmz_sg = ec2.SecurityGroup('webDmzSg',
                               GroupDescription='allow ssh and http traffic',
                               SecurityGroupIngress=rules,
                               VpcId=Ref(vpc),
                               Tags=rename_me)
resources.add(web_dmz_sg)

### Routes

In [8]:
route_table = ec2.RouteTable('routeTable', VpcId=Ref(vpc), Tags=rename_me)
route_to_igw = ec2.Route('routeToIgw',
                         DestinationCidrBlock='0.0.0.0/0',
                         RouteTableId=Ref(route_table),
                         GatewayId=Ref(igw))
associate_route_to_public1 = ec2.SubnetRouteTableAssociation('associateToPublic1', 
                                                             RouteTableId=Ref(route_table),
                                                             SubnetId=Ref(public1))
associate_route_to_public2 = ec2.SubnetRouteTableAssociation('associateToPublic2', 
                                                             RouteTableId=Ref(route_table),
                                                             SubnetId=Ref(public2))
associate_route_to_public3 = ec2.SubnetRouteTableAssociation('associateToPublic3', 
                                                             RouteTableId=Ref(route_table),
                                                             SubnetId=Ref(public3))
resources.add(route_table)
resources.add(route_to_igw)
resources.add(associate_route_to_public1)
resources.add(associate_route_to_public2)
resources.add(associate_route_to_public3)

## LoadBalancing

## AutoScaling

## Bastion

# Outputs

#  Build
---

In [9]:
file_name = 'custom_monitoring.json'

def build(filename):
    fname = 'outputs/' + filename
    for r in resources:
        t.add_resource(r)
    
    with open(fname, 'w') as fd:
        fd.write(t.to_json())
    
    !aws cloudformation validate-template --template-body file://{fname} > /dev/null && echo "build ok"

build(file_name)

ok
